In [ ]:
# written by Tsai Tim
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt 
import seaborn as seabornInstance 
import seaborn as sns
from sklearn.model_selection import train_test_split 
from xgboost import XGBRegressor
import lightgbm as lgb
import numpy  
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import statsmodels.api as sm
%matplotlib inline
import pickle
from sklearn import metrics
from sklearn.metrics import r2_score

In [ ]:
train_data = pd.read_csv("file")

X = train_data[train_data.columns[1:20]].values
y = train_data[train_data.columns[0]].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## light GBM

In [ ]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l2', 'auc'],
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 1000,
    "n_estimators": 1000
}

gbm = lgb.LGBMRegressor(**hyper_params)

## 訓練GBM模型

In [ ]:
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=1000)
#存model
pickle.dump(gbm, open("LGBM.pickle.dat", "wb"))

## 讀model

In [ ]:
lgbm_loaded_model = pickle.load(open("LGBM.pickle.dat", "rb"))

## 預測

In [ ]:
test_y_predicted = lgbm_loaded_model.predict(X_test, num_iteration=lgbm_loaded_model.best_iteration_)

## XGB參數選取

In [ ]:
xgb1 = XGBRegressor()
# parameters = {'nthread':[4], # XGBRegressor 參數測試
#               'objective':['reg:linear'],
#               'learning_rate': [.03, 0.05, .07], #so called `eta` value
#               'max_depth': [5, 6, 7],
#               'min_child_weight': [4],
#               'silent': [1],
#               'subsample': [0.7],
#               'colsample_bytree': [0.7],
#               'n_estimators': [500]}

parameters = {'nthread':[4],
              'objective':['reg:linear'],
              'learning_rate': [.07],
              'max_depth': [7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

## 訓練XGB 模型

In [ ]:
xgb_grid.fit(X_train,
         y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

# 存 model
pickle.dump(xgb_grid, open("xgb.pickle.dat", "wb"))

## 讀model

In [ ]:
xgb_loaded_model = pickle.load(open("xgb.pickle.dat", "rb"))

## 預測

In [ ]:
test_y_predicted = xgb_loaded_model.predict(X_test)

print(X_test.shape)
print(test_y_predicted.shape)
print(y_test.shape)

In [ ]:
df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': test_y_predicted.flatten()})
print(df)

In [ ]:
#線性圖
df1 = df.head(200)
df1.plot(figsize = (20,8))
plt.show()

## 實際與偵測值的error差別

In [ ]:
plt.plot(y_test-test_y_predicted,label='error')
plt.legend()

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, test_y_predicted))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, test_y_predicted))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, test_y_predicted)))

## 算出r2 square

In [ ]:
r2_final = r2_score(y_test, test_y_predicted)
plt.scatter(y_test,test_y_predicted,color="#d62728")
plt.xlabel('Actual values')
plt.ylabel('Predicted values')

plt.plot(np.unique(y_test), np.poly1d(np.polyfit(y_test, test_y_predicted, 1))(np.unique(y_test)))

plt.text(20000, 1, 'R-squared = %0.2f' % r2_final)